In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 4;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
global Transition_matrix = 
            [0.8571 0.1429 0 0;
        0 0.8571 0.1429 0;
        0 0 0.8 0.2
      0 0 0 1];
Transition_matrix=Transition_matrix./[sum(Transition_matrix[i,:]) for i in 1:4]
failure_penalty = -1000.0;
maintenance_penalty = -200.0;
setup_cost = -800.0;
normal_operation = 0.0;

In [3]:
action=[(0,0),(0,-1),(1,-1)]


3-element Vector{Tuple{Int64, Int64}}:
 (0, 0)
 (0, -1)
 (1, -1)

In [4]:
action

3-element Vector{Tuple{Int64, Int64}}:
 (0, 0)
 (0, -1)
 (1, -1)

In [5]:
# function reward(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
#        if s==Number_level
#         return (failure_penalty+normal_operation +  setup_cost/NumberUnits)
#        end
#         r=0
#      if a==1
            
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation ;
#          return r
#         end
#     if a==2
            
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation +  setup_cost/NumberUnits;
#          return r
#         end
#     if a==3
            
#             s=1
#         r = maintenance_penalty +  setup_cost/NumberUnits
#         return r
#         end
#     if a>3 && a <=NumberUnits+3
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation ;
#              return r
#         end
#     if a>NumberUnits+3
#         s=1
#         r = maintenance_penalty +normal_operation+ setup_cost/action[a][2]
#         return r
#     end
#     end

In [6]:
function reward(s, a,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
       if s==Number_level
            r=failure_penalty +  setup_cost/NumberUnits  + normal_operation  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
    if a==2
         if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits;
         return r
        else
            return 0.0
        end
        end
    if a==3
        if s==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= normal_operation +  setup_cost/NumberUnits ;
            return r
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
        return r
        end
    end
    end

reward (generic function with 3 methods)

In [7]:
# function reward(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
#        if s==Number_level
#         s=1
# #         crd = Categorical(Transition_matrix[s,:]);
# #             s = rand(crd);
#             r=failure_penalty +  setup_cost/NumberUnits # + normal_operation  ;
#         return r
#        end
#         r=0
#      if a==1
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation ;
#          return r
#         end
#     if a==2
#             s1=deepcopy(s)
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation +  setup_cost/NumberUnits ;
# #              r*=(1-(s1/10));
#              return r
#         end
#     if a==3
#         s1=deepcopy(s)
#         s=1
#         r = maintenance_penalty +  setup_cost/NumberUnits
# #          crd = Categorical(Transition_matrix[s,:]);
# #             s = rand(crd);
# #             r+= normal_operation  ;
# #            r*=(s1/10);
#         return r
#     end
#     end

In [8]:
s=1:Number_level;

In [9]:
(Transition_matrix)

4×4 Matrix{Float64}:
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.0     0.0     0.0     1.0

In [10]:
T=zeros(Number_level,Number_level,3);


In [11]:
((1:10)./10)

0.1:0.1:1.0

In [12]:
T[:,:,1].=Transition_matrix;

In [13]:
T[:,:,2].=Transition_matrix;

In [14]:
for j in 1:4
        T[j,:,3].=Transition_matrix[1,:];
        end
# for j in 1:10
#         T[j,:,3].=0.0
#     T[j,1,3]=1.0
#         end

In [15]:
T[4,:,1].=Transition_matrix[1,:]
T[4,:,2].=Transition_matrix[1,:]

4-element view(::Array{Float64, 3}, 4, :, 2) with eltype Float64:
 0.8571
 0.1429
 0.0
 0.0

In [16]:
# T[:,:,1].=T[:,:,1].*(repeat(.5:.5,10)-((10:1)./20))
# T[:,:,2].=T[:,:,2].*(repeat(.5:.5,10)-((10:1)./20))

In [17]:
T

4×4×3 Array{Float64, 3}:
[:, :, 1] =
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.8571  0.1429  0.0     0.0

[:, :, 2] =
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.8571  0.1429  0.0     0.0

[:, :, 3] =
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0

In [18]:
T[:, :, 3]

4×4 Matrix{Float64}:
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0

In [19]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3
        if s==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= normal_operation+  setup_cost/NumberUnits  ;
            return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
        end
    end
    end

generative (generic function with 2 methods)

In [20]:
function simulate!(V,a,T, s,S,γ, d)
    if d ≤ 0
        return sum(T[s,s1,1]*V[s1] for (s1) in S)
    end
#     a = explore()
#     print("IN")

    s′, r = generative(s,a)
    q = r + γ*simulate!(V,a,T, s′,S,γ, d-1)
#     print("out")
#     N[s,a] += 1
#     Q[s,a] += (q-Q[s,a])/N[s,a]
    return q
end

simulate! (generic function with 1 method)

In [21]:
T_a=[0.3,0.4,0.3]

3-element Vector{Float64}:
 0.3
 0.4
 0.3

In [22]:
1-(0.5-(1/(10*9)))^10

0.9992199873060514

In [23]:
[-810.2754441030092 -1022.5068576184068 -1465.2034672077405; -795.7942259663222 -995.1156632238184 -1323.545947966713; -855.6459507544228 -1033.393219485689 -1290.8725471675034; -909.7887259606524 -1078.4902024448909 -1279.6914040535578; -987.9666610116174 -1149.3447118709403 -1269.2243767561583; -1016.1112913989151 -1235.7506405492472 -1291.067258596298; -1198.6165653126066 -1439.519296024197 -1196.1981257112018; -1433.7668360760463 -1669.2129697123328 -1137.6574666552951; -2329.8056330773397 -2507.058759945205 -1112.3193274102573; -2961.2724440666325 -2988.459896148223 -2977.771882937651]

10×3 Matrix{Float64}:
  -810.275  -1022.51   -1465.2
  -795.794   -995.116  -1323.55
  -855.646  -1033.39   -1290.87
  -909.789  -1078.49   -1279.69
  -987.967  -1149.34   -1269.22
 -1016.11   -1235.75   -1291.07
 -1198.62   -1439.52   -1196.2
 -1433.77   -1669.21   -1137.66
 -2329.81   -2507.06   -1112.32
 -2961.27   -2988.46   -2977.77

In [24]:
[0.4199284380833852 0.3406813939016802 0.23939016801493465; 0.39208335487874246 0.3376079424996122 0.27030870262164536; 0.37193903421123276 0.3378172815939392 0.29024368419482804; 0.3739568622416228 0.33213506226730005 0.29390807549107717; 0.36880953027054897 0.32853925266614203 0.302651217063309; 0.36975563687876284 0.3207885727777122 0.309455790343525; 0.3556647367171376 0.2906613577834195 0.35367390549944294; 0.3253475414141793 0.2616925876592312 0.4129598709265895; 0.25193825280354426 0.22064931468918733 0.5274124325072684; 0.3305978898007034 0.33571913370765716 0.3336829764916394]


10×3 Matrix{Float64}:
 0.419928  0.340681  0.23939
 0.392083  0.337608  0.270309
 0.371939  0.337817  0.290244
 0.373957  0.332135  0.293908
 0.36881   0.328539  0.302651
 0.369756  0.320789  0.309456
 0.355665  0.290661  0.353674
 0.325348  0.261693  0.41296
 0.251938  0.220649  0.527412
 0.330598  0.335719  0.333683

In [25]:
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,3))
#     print(Q)
    V= rand(Float64, (Number_level))*1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    while i<=2000
#     while  i<=10000
    for s in 1:4
        Threads.@threads for a in 1:3
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:4 ))
    end

           p=[exp(lamb*(sum(Q[s,1]-findmax(Q[s,1:3])[1]))),exp(lamb*(Q[s,2]-findmax(Q[s,1:3])[1])),exp(lamb*(Q[s,3]-findmax(Q[s,1:3])[1]))]

            p=p./sum(p)
            V[s]=Q[s,1]*p[1]+Q[s,2]*p[2]+Q[s,3]*p[3]
        end
        i+=1
        end
    return Q,V
    end

ValueIteration (generic function with 3 methods)

In [26]:
function ValueIteration2(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,3))
#     print(Q)
    V= rand(Float64, (Number_level))*1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    while i<=1500
#     while  i<=10000
    for s in 1:4
        Threads.@threads for a in 1:3
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:4 ))
#                 Q[s,a]= ( sum(T[s,s1,a]*(reward(s,a,s1) + γ*(V[s1])) for (s1) in S ))

#                 Q[s,a]= ( sum(T[s,s1,a]*(reward(s,a,s1) + γ*(0.2*findmax(Q[s1,1])[1]+0.8*findmax(Q[s1,2:3])[1])) for (s1) in S ))
        end
#      temp=V[s]
#          V[:].=(findmax(((Q[s1,1]*p+Q[s1,2]*(1-p)), Q[s1,3]))[1] for s1 in S)
#       V[:,2].=(findmax(Q[s1,2:3])[1] for s1 in S)
#             println(exp(Q[s,1]))
            p=[exp(lamb*(sum(Q[s,1]-findmax(Q[s,1:3])[1]))),exp(lamb*(Q[s,2]-findmax(Q[s,1:3])[1])),exp(lamb*(Q[s,3]-findmax(Q[s,1:3])[1]))]
#                p=[sum(exp(lamb*Q[s1,1]) for s1 in 1:Number_level),
#                 sum(exp(lamb*Q[s1,2]) for s1 in 1:Number_level),
#                 sum(exp(lamb*Q[s1,3]) for s1 in 1:Number_level)]

            p=p./sum(p)
#             p=p./sum(p)
#             println((p))
       V[s]=findmax(Q[s,1:3])[1]
        #     V[s]=Q[s,1]*p[1]+max(Q[s,2],Q[s,3])*(p[2]+p[3])
# println(( V[s]))
        end
        i+=1
        end
    return Q,V
    end

ValueIteration2 (generic function with 3 methods)

In [27]:
NumberUnits=10
lamb=0.1

Q,V=ValueIteration(s,T,.95,0.01,3000)

([-321.7321925056723 -401.7321925056723 -401.7321925056723; -439.8809975921638 -519.8809975921638 -601.7321925056723; -723.646053265966 -803.646053265966 -601.7321925056723; -1401.7321925056722 -1401.7321925056722 -1401.7321925056722], [-321.78583053902554, -439.9078407309011, -601.7328114340636, -1401.7321925056722])

In [28]:
Q[:,3]

4-element Vector{Float64}:
  -401.7321925056723
  -601.7321925056723
  -601.7321925056723
 -1401.7321925056722

In [29]:
V

4-element Vector{Float64}:
  -321.78583053902554
  -439.9078407309011
  -601.7328114340636
 -1401.7321925056722

In [30]:
NumberUnits=10
Q2,V2=ValueIteration2(s,T,.95,0.01,1000)

([-321.00013106901673 -401.00013106901673 -401.00013106901673; -439.2278689309805 -519.2278689309805 -601.0001310690168; -722.9501245155659 -802.9501245155659 -601.0001310690168; -1401.0001310690168 -1401.0001310690168 -1401.0001310690168], [-321.00013106901673, -439.2278689309805, -601.0001310690168, -1401.0001310690168])

In [31]:
Q2[:,3]


4-element Vector{Float64}:
  -401.00013106901673
  -601.0001310690168
  -601.0001310690168
 -1401.0001310690168

In [32]:
V2

4-element Vector{Float64}:
  -321.00013106901673
  -439.2278689309805
  -601.0001310690168
 -1401.0001310690168

In [33]:

# function decison(Q, s)
#    a_final=[]
#    if Number_level in s
# #         print("hi")
#         d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#         d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
#         min=-1e+10
#         opt=0
#         for (i,j) in enumerate(d)
#             if min < sum((j))
#                 min=sum((j))
#                 opt=i
#             end
#         end
#         return d1[opt]
#     end
#     q0=0.0
#     a_0=repeat(1:1,NumberUnits);
#     for i in 1:NumberUnits
#             q0+=Q[s[i],1]
#     end
#     d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#     d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
# #     d[1]=a_0
# #     print("here")
# #     d1[1]=(1,1,1,1,1,1,1,1,1,1)
#     min=-1e+10
#     opt=0
#     for (i,j) in enumerate(d)
#         if i==1
#             if min < q0
#             min=q0
#             opt=i
#         end
#         elseif min < sum((j))
#             min=sum((j))
#             opt=i
#         end
#     end
#     if opt==1
#         return a_0
#     else
#         return d1[opt]
#     end
#     return opt
# end


function decison(Q, s,n=0.2)
   a_final=[]
   if Number_level in s
#         print("hi")
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a], [2,3]))
            end
        return a_final
    end
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1]
    end
    a_final=repeat(1:1,NumberUnits);
    repair= argmax(i->s[i], 1:NumberUnits)
#     print(repair)
    a_final[repair]=3
    b=1
    for i in 1:NumberUnits
            if i!=repair
            a_final[i]=argmax(a->Q[s[i],a], [2,3])
#             println(s[i])
#             println(a_final[i])

            if a_final[i]==3
                b+=1
            end
            end
    end
        q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_final[i]]
    end
#     if b<=NumberUnits*n
#         return a_0
#     end
    if q0>q1
            return a_0
        else
            return a_final
        end
end

decison (generic function with 2 methods)

In [34]:
decison(Q,[8,8
        ,8,8
        ,1,1,6,1, 7,7,8,8
        ,8,8
        ,1,1,6,1, 7,7])

LoadError: BoundsError: attempt to access 4×3 Matrix{Float64} at index [8, 1]

In [35]:
using DataFrames
df= DataFrame(Units=[],n=[],s=[],m=[],f=[],lambda=[],mean=[],std=[])

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any


In [36]:
global NumberUnits=3   
global lamb=0.001694915254237288
    
     Q,V=ValueIteration(s,T,.95,3000)

([-3330.6000588110805 -3597.266725477747 -3597.266725477747; -3507.1959898870673 -3773.862656553734 -3797.266725477747; -3816.808112307911 -4083.4747789745775 -3797.266725477747; -4597.266725477747 -4597.266725477747 -4597.266725477747], [-3479.933899571344, -3661.60569227367, -3872.799255877815, -4597.266725477746])

In [37]:
cost=[ 
     [0 -200 -200 -1000],
    [0 -400 -200 -1000],
    [0 -600 -200 -1000],
    [0 -800 -200 -1000],
    [0 -1000 -200 -1000],
    [0 -1200 -200 -1000],
    [0 -1400 -200 -1000],
    [0 -1600 -200 -1000]
    ]


for units in [3,5,10,15,20,25,30,40,50,60,70,80,100,125,150,175,200]
    for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in [100,10,5,1,0.5, 0.1,0.05,1e-2,0.5e-2,1e-3,0.5e-3,1e-4,0.5e-4,1e-5,0.5e-5,1e-6,0.5e-6,1e-7,0.5e-7,1e-8]
        global  lamb=lambda
    global NumberUnits=units
     Q,V=ValueIteration(s,T,.95,3000)
         println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
#     print(a)
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
         s[i],r[i]=generative(s[i],a[i],234);
          r2+=r[i]
            end 
#     println(r2)
#     println(s)
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

3[0, -200, -200, -1000]
false


[ Info:  started timer at: 2023-03-24T10:13:18.172


3
100.0
-1780.308538307025
575.8158657499376


[ Info:            2.9919039s: 2 seconds, 991 milliseconds


false


...

false


[ Info:  started timer at: 2023-03-25T02:25:33.652


125
1.0e-6
-42090.688847515354
1398.0100296469284


[ Info:           94.7889492s: 1 minute, 34 seconds, 788 milliseconds
Excessive output truncated after 524292 bytes.

false


In [38]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,200.0,0.0,-1600.0,-200.0,-1000.0,1.0e-8,-72876.6,1864.81
2,200.0,0.0,-1600.0,-200.0,-1000.0,5.0e-8,-72856.4,1863.36
3,200.0,0.0,-1600.0,-200.0,-1000.0,1.0e-7,-72838.8,1863.92
4,200.0,0.0,-1600.0,-200.0,-1000.0,5.0e-7,-72847.0,1860.68
5,200.0,0.0,-1600.0,-200.0,-1000.0,1.0e-6,-72874.8,1848.82
6,200.0,0.0,-1600.0,-200.0,-1000.0,5.0e-6,-72829.2,1845.69
7,200.0,0.0,-1600.0,-200.0,-1000.0,1.0e-5,-72847.5,1857.72
8,200.0,0.0,-1600.0,-200.0,-1000.0,5.0e-5,-72830.9,1870.03
9,200.0,0.0,-1600.0,-200.0,-1000.0,0.0001,-72800.3,1868.25


In [39]:
using CSV
CSV.write("valueiteration_softmax_space_division.csv", df)

"valueiteration_softmax_space_division.csv"

In [40]:
 Q=ValueIteration(s,T,.95,3000)
for s in 1:Number_level
p=[exp(0.01*sum(Q[s,1])),exp(0.01*sum(Q[s,2])),exp(0.01*sum(Q[s,3]))]
    p./=sum(p)
    println(p)
end

LoadError: MethodError: no method matching getindex(::Tuple{Matrix{Float64}, Vector{Float64}}, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  getindex(::Tuple, ::Int64) at tuple.jl:29
[0m  getindex(::Tuple, ::Integer) at tuple.jl:30
[0m  getindex(::Tuple, ::Real) at deprecated.jl:70
[0m  ...

In [41]:
using Plots

LoadError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.

In [42]:
for i in [3,5,10,15,20,25,30,40,50,60,70,80,100,125,150,175,200]
   display( plot(df[df.Units.==i,:].lambda,df[df.Units.==i,:].mean, xtick=df[df.Units.==i,:].lambda, show = true,title=string(i),label=false, xaxis=:log10,rotation=40))
    plot(show=true)
    println(df[df.Units.==i,:][findmax(df[df.Units.==i,:].mean)[2],:])

end

LoadError: UndefVarError: plot not defined

In [43]:
plot!()


LoadError: UndefVarError: plot! not defined